# Aadhaar Data Cleaning & Preparation

Welcome! This notebook is designed to help to process and clean Aadhaar datasets (Biometric, Demographic, and Enrolment) easily.

### What this notebook does:
1. **Loads** data from multiple CSV files.
2. **Merges** them into one large dataset for each category.
3. **Cleans** the state names (fixes spelling mistakes like 'Telengana' -> 'Telangana').
4. **Saves** the clean data to new files for analysis.

In [23]:
import pandas as pd
import numpy as np
import os
from IPython.display import display  # To show DataFrames nicely

# Display settings to show all columns
pd.set_option('display.max_columns', None)

## Step 1: Configuration
We define a list of **correct state names** and a **dictionary** to fix common spelling errors.

In [24]:
# CORRECT SPELLING -> Standard Name
STATE_MAPPING = {
    'westbengal': 'West Bengal',
    'west bangal': 'West Bengal',
    'west bengli': 'West Bengal',
    'wb': 'West Bengal',
    'west  bengal': 'West Bengal',
    'odisha': 'Odisha',
    'orissa': 'Odisha',
    'pondicherry': 'Puducherry',
    'puducherry': 'Puducherry',
    'uttaranchal': 'Uttarakhand',
    'uttarakhand': 'Uttarakhand',
    'jammu & kashmir': 'Jammu & Kashmir',
    'jammu and kashmir': 'Jammu & Kashmir',
    'andaman & nicobar islands': 'Andaman & Nicobar Islands',
    'andaman and nicobar islands': 'Andaman & Nicobar Islands',
    'chhatisgarh': 'Chhattisgarh',
    'chhattisgarh': 'Chhattisgarh',
    'tamilnadu': 'Tamil Nadu',
    'tamil nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'dadra and nagar haveli': 'Dadra & Nagar Haveli and Daman & Diu',
    'dadra & nagar haveli': 'Dadra & Nagar Haveli and Daman & Diu',
    'daman and diu': 'Dadra & Nagar Haveli and Daman & Diu',
    'daman & diu': 'Dadra & Nagar Haveli and Daman & Diu',
    'dadra and nagar haveli and daman and diu': 'Dadra & Nagar Haveli and Daman & Diu',
    'the dadra and nagar haveli and daman and diu': 'Dadra & Nagar Haveli and Daman & Diu'
}

# List of valid states to keep (Anything else will be removed as garbage data)
VALID_STATES = {
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh',
    'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka',
    'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram',
    'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu',
    'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal',
    'Andaman & Nicobar Islands', 'Chandigarh', 'Dadra & Nagar Haveli and Daman & Diu',
    'Delhi', 'Jammu & Kashmir', 'Ladakh', 'Lakshadweep', 'Puducherry'
}

## Step 2: Helper Functions
These functions handle the repetitive work of loading files and cleaning the state column.

In [25]:
def load_and_merge(file_list, output_raw_path=None):
    """Loads multiple CSV files and merges them into a single DataFrame."""
    print(f"Loading {len(file_list)} files...")
    dfs = []
    for f in file_list:
        if os.path.exists(f):
            print(f"  - Reading {f}")
            dfs.append(pd.read_csv(f))
        else:
            print(f"  - Warning: File not found: {f}")
    
    if not dfs:
        print("No files loaded!")
        return pd.DataFrame()
        
    merged = pd.concat(dfs, axis=0, ignore_index=True)
    print(f"Total rows loaded: {len(merged)}")
    
    if output_raw_path:
        merged.to_csv(output_raw_path, index=False)
        print(f"Saved raw combined data to: {output_raw_path}")
        
    return merged

def clean_state_column(df, state_col='state'):
    """Standardizes state names and removes invalid entries."""
    if df.empty: return df
    
    print("Cleaning state column...")
    initial_count = len(df)
    
    # 1. Basic cleanup (remove spaces, lowercase)
    df[state_col] = df[state_col].astype(str).str.strip().str.lower()
    df[state_col] = df[state_col].str.replace(r'\s+', ' ', regex=True)
    
    # 2. Fix spelling using our mapping
    df[state_col] = df[state_col].replace(STATE_MAPPING)
    
    # 3. Capitalize nicely (Title Case)
    df[state_col] = df[state_col].str.title()
    
    # 4. Remove invalid states
    df = df[df[state_col].isin(VALID_STATES)]
    
    final_count = len(df)
    rows_dropped = initial_count - final_count
    print(f"Dropped {rows_dropped} rows with invalid state names.")
    print(f"Remaining rows: {final_count}")
    
    return df

--- 
## Step 3: Biometric Data Processing
Files processed: `api_data_aadhar_biometric_0_500000` to `api_data_aadhar_biometric_1500000_1861108`

In [33]:
# 1. Define input files
bio_files = [
    "../data/raw/api_data_aadhar_biometric/api_data_aadhar_biometric_0_500000.csv",
    "../data/raw/api_data_aadhar_biometric/api_data_aadhar_biometric_500000_1000000.csv",
    "../data/raw/api_data_aadhar_biometric/api_data_aadhar_biometric_1000000_1500000.csv",
    "../data/raw/api_data_aadhar_biometric/api_data_aadhar_biometric_1500000_1861108.csv"
]

# 2. Load and Merge
bio_df = load_and_merge(bio_files, "../data/processed/combined_biometric.csv")

# 3. Clean
bio_df = clean_state_column(bio_df)
print("Preview of Cleaned Biometric Data:")
display(bio_df.head())

# 4. Save
bio_df.to_csv("../data/processed/combined_biometric_cleaned.csv", index=False)
print("Biometric data saved to 'combined_biometric_cleaned.csv'")

Loading 4 files...
  - Reading ../data/raw/api_data_aadhar_biometric/api_data_aadhar_biometric_0_500000.csv
  - Reading ../data/raw/api_data_aadhar_biometric/api_data_aadhar_biometric_500000_1000000.csv
  - Reading ../data/raw/api_data_aadhar_biometric/api_data_aadhar_biometric_1000000_1500000.csv
  - Reading ../data/raw/api_data_aadhar_biometric/api_data_aadhar_biometric_1500000_1861108.csv
Total rows loaded: 1861108
Saved raw combined data to: ../data/processed/combined_biometric.csv
Cleaning state column...
Dropped 1325 rows with invalid state names.
Remaining rows: 1859783
Preview of Cleaned Biometric Data:


,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,Haryana,Mahendragarh,123029,280,577
1,01-03-2025,Bihar,Madhepura,852121,144,369
2,01-03-2025,Jammu & Kashmir,Punch,185101,643,1091
3,01-03-2025,Bihar,Bhojpur,802158,256,980
4,01-03-2025,Tamil Nadu,Madurai,625514,271,815


Biometric data saved to 'combined_biometric_cleaned.csv'


--- 
## Step 4: Demographic Data Processing
Files processed: `api_data_aadhar_demographic_0_500000` to `api_data_aadhar_demographic_2000000_2071700`

In [34]:
# 1. Define input files
demo_files = [
    "../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_0_500000.csv",
    "../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_500000_1000000.csv",
    "../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_1000000_1500000.csv",
    "../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_1500000_2000000.csv",
    "../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_2000000_2071700.csv"
]

# 2. Load and Merge
demo_df = load_and_merge(demo_files, "../data/processed/combined_demographic.csv")

# 3. Clean
demo_df = clean_state_column(demo_df)
print("Preview of Cleaned Demographic Data:")
display(demo_df.head())

# 4. Save
demo_df.to_csv("../data/processed/combined_demographic_cleaned.csv", index=False)
print("Demographic data saved to 'combined_demographic_cleaned.csv'")

Loading 5 files...
  - Reading ../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_0_500000.csv
  - Reading ../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_500000_1000000.csv
  - Reading ../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_1000000_1500000.csv
  - Reading ../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_1500000_2000000.csv
  - Reading ../data/raw/api_data_aadhar_demographic/api_data_aadhar_demographic_2000000_2071700.csv
Total rows loaded: 2071700
Saved raw combined data to: ../data/processed/combined_demographic.csv
Cleaning state column...
Dropped 1640 rows with invalid state names.
Remaining rows: 2070060
Preview of Cleaned Demographic Data:


,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,Uttar Pradesh,Gorakhpur,273213,49,529
1,01-03-2025,Andhra Pradesh,Chittoor,517132,22,375
2,01-03-2025,Gujarat,Rajkot,360006,65,765
3,01-03-2025,Andhra Pradesh,Srikakulam,532484,24,314
4,01-03-2025,Rajasthan,Udaipur,313801,45,785


Demographic data saved to 'combined_demographic_cleaned.csv'


--- 
## Step 5: Enrolment Data Processing
Files processed: `api_data_aadhar_enrolment_0_500000` to `api_data_aadhar_enrolment_1000000_1006029`

In [35]:
# 1. Define input files
enrol_files = [
    "../data/raw/api_data_aadhar_enrolment/api_data_aadhar_enrolment_0_500000.csv",
    "../data/raw/api_data_aadhar_enrolment/api_data_aadhar_enrolment_500000_1000000.csv",
    "../data/raw/api_data_aadhar_enrolment/api_data_aadhar_enrolment_1000000_1006029.csv"
]

# 2. Load and Merge
enrol_df = load_and_merge(enrol_files, "../data/processed/combined_enrolment.csv")

# 3. Clean
enrol_df = clean_state_column(enrol_df)
print("Preview of Cleaned Enrolment Data:")
display(enrol_df.head())

# 4. Save
enrol_df.to_csv("../data/processed/combined_enrolment_cleaned.csv", index=False)
print("Enrolment data saved to 'combined_enrolment_cleaned.csv'")

Loading 3 files...
  - Reading ../data/raw/api_data_aadhar_enrolment/api_data_aadhar_enrolment_0_500000.csv
  - Reading ../data/raw/api_data_aadhar_enrolment/api_data_aadhar_enrolment_500000_1000000.csv
  - Reading ../data/raw/api_data_aadhar_enrolment/api_data_aadhar_enrolment_1000000_1006029.csv
Total rows loaded: 1006029
Saved raw combined data to: ../data/processed/combined_enrolment.csv
Cleaning state column...
Dropped 438 rows with invalid state names.
Remaining rows: 1005591
Preview of Cleaned Enrolment Data:


,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,02-03-2025,Meghalaya,East Khasi Hills,793121,11,61,37
1,09-03-2025,Karnataka,Bengaluru Urban,560043,14,33,39
2,09-03-2025,Uttar Pradesh,Kanpur Nagar,208001,29,82,12
3,09-03-2025,Uttar Pradesh,Aligarh,202133,62,29,15
4,09-03-2025,Karnataka,Bengaluru Urban,560016,14,16,21


Enrolment data saved to 'combined_enrolment_cleaned.csv'


In [ ]:
---
## Step 6: Backend Data Preparation for Aadhaar Trends Dashboard
This script implements a robust ETL (Extract, Transform, Load) workflow to prepare raw Aadhaar transaction logs 
for downstream analysis and visualization. It handles data standardization, temporal parsing, and feature 
engineering to create a unified 'Master Dataset'

In [38]:
# 1. LOAD DATASETS

enrol = pd.read_csv("../data/processed/combined_enrolment_cleaned.csv")
bio   = pd.read_csv("../data/processed/combined_biometric_cleaned.csv")
demo  = pd.read_csv("../data/processed/combined_demographic_cleaned.csv")

# 2. CLEAN TEXT COLUMNS

for df in [enrol, bio, demo]:
    df['state'] = df['state'].astype(str).str.strip().str.title()
    df['district'] = df['district'].astype(str).str.strip().str.title()
    df['pincode'] = df['pincode'].astype(str).str.strip()

# 3. FIX DATE FORMAT (DD-MM-YYYY)

enrol['date'] = pd.to_datetime(enrol['date'], dayfirst=True, errors='coerce')
bio['date']   = pd.to_datetime(bio['date'], dayfirst=True, errors='coerce')
demo['date']  = pd.to_datetime(demo['date'], dayfirst=True, errors='coerce')

# 4. MERGE DATASETS

df = enrol.merge(
    bio,
    on=['date', 'state', 'district', 'pincode'],
    how='outer'
)

df = df.merge(
    demo,
    on=['date', 'state', 'district', 'pincode'],
    how='outer'
)


# 5. HANDLE MISSING VALUES

df.fillna(0, inplace=True)

# 6. FEATURE ENGINEERING


# Total Aadhaar Enrolment
df['Total_Enrolment'] = (
    df['age_0_5'] +
    df['age_5_17'] +
    df['age_18_greater']
)

# Total Aadhaar Updates
df['Total_Updates'] = (
    df['bio_age_5_17'] +
    df['bio_age_17_'] +
    df['demo_age_5_17'] +
    df['demo_age_17_']
)

# Youth Activity Share 
df['Youth_Share'] = np.where(
    (df['Total_Enrolment'] + df['Total_Updates']) > 0,
    (
        (df['age_5_17'] + df['bio_age_5_17'] + df['demo_age_5_17']) /
        (df['Total_Enrolment'] + df['Total_Updates'])
    ) * 100,
    0
)

# 7. OPTIONAL: SORT DATA

df.sort_values(by=['date', 'state', 'district'], inplace=True)


# 8. SAVE FINAL DATASET

df.to_csv("../data/processed/master_data.csv", index=False)

print("master_data.csv created successfully and ready for Tableau!")

master_data.csv created successfully and ready for Tableau!
